다 대 일(many-to-one) 구조의 RNN을 사용하여 문맥을 반영해서 텍스트를 생성하는 모델을 만들어보자!

# 1. RNN을 이용하여 텍스트 생성하기
---

예를 들어서 '경마장에 있는 말이 뛰고 있다'와 '그의 말이 법이다'와 '가는 말이 고와야 오는 말이 곱다'라는 세 가지 문장이 있다고 해보자. 모델이 문맥을 학습할 수 있도록, 전체 문장의 앞의 단어들을 전부 고려하여 학습하도록 데이터를 재구성한다면 아래와 같이 총 11개의 샘플이 구성된다. 이 얘기는 앞서 [section 5의 RNN 언어 모델](https://github.com/yhyuntak/STUDY_AI/blob/main/Tensorflow/%E1%84%8E%E1%85%A2%E1%86%A8_%E1%84%83%E1%85%B5%E1%86%B8_%E1%84%85%E1%85%A5%E1%84%82%E1%85%B5%E1%86%BC%E1%84%8B%E1%85%B3%E1%86%AF_%E1%84%8B%E1%85%B5%E1%84%8B%E1%85%AD%E1%86%BC%E1%84%92%E1%85%A1%E1%86%AB_%E1%84%8C%E1%85%A1%E1%84%8B%E1%85%A7%E1%86%AB%E1%84%8B%E1%85%A5_%E1%84%8E%E1%85%A5%E1%84%85%E1%85%B5_%E1%84%8B%E1%85%B5%E1%86%B8%E1%84%86%E1%85%AE%E1%86%AB/CH8.%20%EC%88%9C%ED%99%98%20%EC%8B%A0%EA%B2%BD%EB%A7%9D(RNN)/section5_RNN%20%EC%96%B8%EC%96%B4%20%EB%AA%A8%EB%8D%B8.ipynb)에서 **교사 강요(teacher forcing)**의 방식으로 학습한다고 배웠다. 현재 시점 t의 단어 하나를 예측하기 위해서 t-1 까지의 모든 시점이 사용되는 것이 포인트다. 

![그림 1](./images/section6/그림_1.png)

## 1) 데이터에 대한 이해와 전처리

먼저 예제로 언급한 3개의 한국어 문장을 변수에 저장하자. 그리고 Tokenizer()로 단어 집합을 생성하고 크기를 확인해보자. 단어 집합의 크기를 저장할 때는 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만, 패딩을 위한 0을 고려하여 +1을 해준다.

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다"""
print("문장들 : ")
print(text)
print("--"*10)
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print('토크나이저의 정수 인코딩 : ',tokenizer.word_index)
print("--"*10)
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 위한 0을 고려해 +1을 하기.
print('단어 집합의 크기 : %d' % vocab_size)




문장들 : 
경마장에 있는 말이 뛰고 있다

그의 말이 법이다

가는 말이 고와야 오는 말이 곱다
--------------------
토크나이저의 정수 인코딩 :  {'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}
--------------------
단어 집합의 크기 : 12


#### 이제 훈련 데이터를 만들어보자.

In [3]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0] # 문장의 단어들을 인덱스화
    for i in range(1, len(encoded)):
        # 표처럼 만들기 위해서 다음과 같이 코딩
        sequence = encoded[:i+1] 
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))
print('샘플들 : ',sequences)

학습에 사용할 샘플의 개수: 11
샘플들 :  [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터이다. [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당된다. 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 한다.

우선 전체 샘플에 대해서 길이를 일치시켜 준다. 가장 긴 샘플의 길이를 기준으로 한다. 현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6이다. 이를 코드로는 다음과 같이 구할 수 있다.

In [4]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


전체 훈련 데이터에서 가장 긴 샘플의 길이가 6임을 확인했다. 전체 샘플의 길이를 6으로 패딩하자.

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("패딩 전 샘플들 :")
print(sequences)
print("--"*10)
sequences = pad_sequences(sequences,maxlen=max_len,padding='pre')

print("패딩 후 샘플들 :")
print(sequences)

패딩 전 샘플들 :
[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]
--------------------
패딩 후 샘플들 :
[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


길이가 6보다 짧은 모든 샘플에 대해서 앞에 0을 채워서 모든 샘플의 길이를 6으로 바꿨다. 이제 각 샘플의 마지막 단어를 레이블로 분리하자. 레이블의 분리는 Numpy를 이용해서 가능하다. 리스트의 마지막 값을 제외하고 저장한 것은 X, 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당된다.

In [6]:
X = sequences[:,:-1]
y = sequences[:,-1]

print("입력 데이터 :")
print(X)
print("--"*10)
print("정답 데이터 :")
print(y)

입력 데이터 :
[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
--------------------
정답 데이터 :
[ 3  1  4  5  1  7  1  9 10  1 11]



RNN 모델에 훈련 데이터를 훈련 시키기 위해 정답 레이블에 대해서 원-핫 인코딩을 적용하자.


In [7]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y,num_classes = vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


아, 입력 데이터에는 원-핫 인코딩을 안하는 구나..?

## 2) 모델 설계하기

RNN 모델에 데이터를 훈련시키도록 하자. 

하이퍼파라미터인 임베딩 벡터의 차원은 10, 은닉 상태의 크기는 32이고 다 대 일 구조의 RNN을 사용한다. 전결합층(Fully Connected Layer)을 출력층으로 단어 집합 크기만큼의 뉴런을 배치하여 모델을 설계한다. 해당 모델은 마지막 시점에서 모든 가능한 단어 중 하나의 단어를 예측하는 다중 클래스 분류 문제를 수행하는 모델이다. 다중 클래스 분류 문제의 경우, 출력층에 소프트맥스 회귀를 사용해야 하므로 활성화 함수로는 소프트맥스 함수를 사용하고, 손실 함수로 크로스 엔트로피 함수를 사용하여 200 에포크를 수행한다.



In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence


Epoch 1/200
1/1 - 1s - loss: 2.4682 - accuracy: 0.0909 - 1s/epoch - 1s/step
Epoch 2/200
1/1 - 0s - loss: 2.4564 - accuracy: 0.2727 - 8ms/epoch - 8ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4447 - accuracy: 0.2727 - 5ms/epoch - 5ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4328 - accuracy: 0.3636 - 5ms/epoch - 5ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4208 - accuracy: 0.4545 - 4ms/epoch - 4ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4085 - accuracy: 0.4545 - 5ms/epoch - 5ms/step
Epoch 7/200
1/1 - 0s - loss: 2.3959 - accuracy: 0.3636 - 5ms/epoch - 5ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3829 - accuracy: 0.4545 - 5ms/epoch - 5ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3694 - accuracy: 0.4545 - 5ms/epoch - 5ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3553 - accuracy: 0.5455 - 4ms/epoch - 4ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3406 - accuracy: 0.5455 - 5ms/epoch - 5ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3252 - accuracy: 0.5455 - 6ms/epoch - 6ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3091 - accuracy: 0.4545 - 5ms/e

Epoch 105/200
1/1 - 0s - loss: 0.5283 - accuracy: 0.8182 - 5ms/epoch - 5ms/step
Epoch 106/200
1/1 - 0s - loss: 0.5174 - accuracy: 0.9091 - 4ms/epoch - 4ms/step
Epoch 107/200
1/1 - 0s - loss: 0.5067 - accuracy: 0.9091 - 7ms/epoch - 7ms/step
Epoch 108/200
1/1 - 0s - loss: 0.4962 - accuracy: 0.9091 - 5ms/epoch - 5ms/step
Epoch 109/200
1/1 - 0s - loss: 0.4860 - accuracy: 0.9091 - 6ms/epoch - 6ms/step
Epoch 110/200
1/1 - 0s - loss: 0.4759 - accuracy: 0.9091 - 5ms/epoch - 5ms/step
Epoch 111/200
1/1 - 0s - loss: 0.4661 - accuracy: 0.9091 - 6ms/epoch - 6ms/step
Epoch 112/200
1/1 - 0s - loss: 0.4565 - accuracy: 0.9091 - 6ms/epoch - 6ms/step
Epoch 113/200
1/1 - 0s - loss: 0.4470 - accuracy: 0.9091 - 6ms/epoch - 6ms/step
Epoch 114/200
1/1 - 0s - loss: 0.4378 - accuracy: 0.9091 - 5ms/epoch - 5ms/step
Epoch 115/200
1/1 - 0s - loss: 0.4288 - accuracy: 0.9091 - 5ms/epoch - 5ms/step
Epoch 116/200
1/1 - 0s - loss: 0.4199 - accuracy: 0.9091 - 6ms/epoch - 6ms/step
Epoch 117/200
1/1 - 0s - loss: 0.4113 - 

In [9]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

print(sentence_generation(model, tokenizer, '그의 말이', 1))

print(sentence_generation(model, tokenizer, '가는', 5))


경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다


앞의 문맥을 기준으로 '말이' 라는 단어 다음에 나올 단어를 기존의 훈련 데이터와 일치하게 예측함을 보여준다. 이 모델은 충분한 훈련 데이터를 갖고 있지 못하므로 위에서 문장의 길이에 맞게 적절하게 예측해야하는 횟수 4, 2, 5를 각각 인자값으로 주었다. 이 이상의 숫자를 주면 기계는 '있다', '법이다', '곱다' 다음에 나오는 단어가 무엇인지 배운 적이 없으므로 임의 예측을 하게 된다.

이번에는 더 많은 훈련 데이터를 가지고 실습해보자.

<br/><br/>
# 2. LSTM을 이용하여 텍스트 생성하기
---

이번에는 LSTM을 통해 보다 많은 데이터로 텍스트를 생성해보겠다.

## 1) 데이터에 대한 이해와 전처리

사용할 데이터는 뉴욕 타임즈 기사의 제목이다. 아래의 링크에서 ArticlesApril2018.csv 데이터를 다운로드하자.
파일 다운로드 링크 : https://www.kaggle.com/aashita/nyt-comments

In [10]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

df = pd.read_csv("./data/ArticlesApril2018.csv")
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   articleID         1324 non-null   object
 1   articleWordCount  1324 non-null   int64 
 2   byline            1324 non-null   object
 3   documentType      1324 non-null   object
 4   headline          1324 non-null   object
 5   keywords          1324 non-null   object
 6   multimedia        1324 non-null   int64 
 7   newDesk           1324 non-null   object
 8   printPage         1324 non-null   int64 
 9   pubDate           1324 non-null   object
 10  sectionName       1324 non-null   object
 11  snippet           1324 non-null   object
 12  source            1324 non-null   object
 13  typeOfMaterial    1324 non-null   object
 14  webURL            1324 non-null   object
dtypes: int64(3), object(12)
memory usage: 155.3+ KB


데이터프레임을 확인해보니, 전부 null 값이 없다. 

이번 예제에선 제목에 해당하는 headline을 사용할 것이므로, headline 열 데이터를 확인해보자.


In [15]:
df['headline'][:5]

0    Former N.F.L. Cheerleaders’ Settlement Offer: ...
1    E.P.A. to Unveil a New Rule. Its Effect: Less ...
2                              The New Noma, Explained
3                                              Unknown
4                                              Unknown
Name: headline, dtype: object

Unknown 이란 값이 존재하는 것을 보았다. 제목이 Unknown이라니.. 좀 이상한 것 같으니 값들의 개수를 확인해보자.

In [17]:
df['headline'].value_counts()

Unknown                                                                        110
Variety: Acrostic                                                                3
Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell      1
As Facebook Loses Luster, Tech Stocks Await Fallout                              1
This Many                                                                        1
                                                                              ... 
Did Outsiders Make 911 Calls? A Fear Born of Brooklyn Gentrification             1
Childhood Fears No Parent Can Allay                                              1
For Bannon, Tariffs Are Test of Trump’s Beliefs                                  1
The Failures of Anti-Trumpism                                                    1
There Is Nothin’ Like a Tune                                                     1
Name: headline, Length: 1213, dtype: int64

무려 Unknown 데이터가 110개나 있다. headline의 데이터 수가 1324인데, 110이면 거의 10%에 해당하는 수준의 결측 값이다. 
따라서 이 unknown 값을 제외하고 headline 값들을 추출하도록 하자.

In [20]:
headline = [title for title in df['headline'] if title != 'Unknown']
print("결측 값을 제거한 headline의 개수 : ",len(headline))

결측 값을 제거한 headline의 개수 :  1214


이제 본격적으로 데이터 전처리를 수행한다. 여기서 선택한 전처리는 구두점 제거와 단어의 소문자화이다. 전처리를 수행하고, 다시 샘플 5개를 출력하도록 하자.

In [22]:
def preprocessing(arr):
    preprocessed_sentence = arr.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    # punctuation에는 ASCII 코드에서 구두점(쉼표, 온점, 따옴표 등) 꾸러미가 들어있다.
    return ''.join(word for word in preprocessed_sentence if word not in punctuation).lower()

preprocessed_headline = [preprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

기존의 출력과 비교하면 모든 단어들이 소문자화되었으며 N.F.L.이나 Cheerleaders’ 등과 같이 기존에 구두점이 붙어있던 단어들에서 구두점이 제거되었다. 그러나 Chapter 2의 section 1에서 배웠듯이, 구두점들을 무조건 삭제하는 것은 옳지 않다. 따라서 원래는 nltk 라이브러리를 이용해서 토큰화를 하는 것이 맞다. 일단은 책의 예시를 그대로 따라가보자.

이제 단어 집합(vocabulary)을 만들고 크기를 확인하자.

In [37]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index)+1 # 
print("단어 집합의 크기 : ",vocab_size)
print("빈도수 상위 10개의 정수 인덱스 : ", [ voca for voca,num in  tokenizer.word_index.items() if num <= 10])


단어 집합의 크기 :  3494
빈도수 상위 10개의 정수 인덱스 :  ['the', 'a', 'to', 'of', 'in', 'for', 'and', 'is', 'on', 'with']


참고로 정수 인덱스는 단어의 빈도수가 높을수록 1에 가깝다. 관사나 전치사가 제일 많은 빈도수를 차지하는 것을 볼 수 있다.

이제 하나의 문장을 제일 위 그림의 표처럼 쪼개서 학습 데이터를 생성하도록 하자. 시퀀스를 2개 3개 ... 계속 붙여나가면서 생성하고 크기를 맞추기 위해 zero padding을 한 후, 제일 마지막 column을 y로 나머지를 X로 사용할 것이다. 왜냐하면 하나의 단어를 예측하기 위해서 이전 시점까지의 모든 데이터를 사용할 것이기 때문이다. 

In [42]:
"""
먼저 한 문장을 여러 시퀀스로 만들어주자.
"""
sequences  = []
for sentence in preprocessed_headline :
    encoded = tokenizer.texts_to_sequences([sentence])[0] # 항상 list 형식으로 문장을 사용해야한다.
    for i in range(1,len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
print("데이터 5개 : ",sequences[:5])

"""
길이가 가장 긴 데이터의 길이를 찾자.
"""
max_len = max(len(temp) for temp in sequences)
print("샘플 데이터의 최대 길이 : ",max_len)

"""
값들 앞에 제로 패딩을 추가하자.
"""
sequences = pad_sequences(sequences,maxlen = max_len, padding='pre')
print("제로 패딩 추가한 데이터 5개 : ")
print(sequences[:5])

"""
입력 데이터 X, 정답 데이터 y 생성
"""
X = sequences[:,:-1]
y = sequences[:,-1]

"""
정답 데이터 y에 원-핫 인코딩 적용
"""
y = to_categorical(y,num_classes = vocab_size)


데이터 5개 :  [[99, 269], [99, 269, 371], [99, 269, 371, 1115], [99, 269, 371, 1115, 582], [99, 269, 371, 1115, 582, 52]]
샘플 데이터의 최대 길이 :  24
제로 패딩 추가한 데이터 5개 : 
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   99  269  371 1115  582]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   99  269  371 1115  582   52]]


## 2) 모델 설계하기

이제 다음 조건의 LSTM 모델을 설계해보자.

* 하이퍼파라미터인 임베딩 벡터의 차원은 10, 은닉 상태의 크기는 128 
* 다 대 일 구조의 LSTM을 사용 
* 전결합층(Fully Connected Layer)을 출력층으로 단어 집합 크기만큼의 뉴런을 배치하여 모델을 설계
* 해당 모델은 마지막 시점에서 모든 가능한 단어 중 하나의 단어를 예측하는 다중 클래스 분류 문제를 수행하는 모델임을 고려해 손실함수, 활성화 함수 설계
* Epoch = 100 (원래 책에선 200이라고 나와있지만, 시간을 절약하기 위해 100으로 줄임)


In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM

embedding_dim = 10
hidden_dim = 128
num_epoch = 100

model = Sequential()
model.add(Embedding(vocab_size,embedding_dim))
model.add(LSTM(hidden_dim,return_sequences = False)) # 원래 False가 default지만 다 대 일 구조를 강조하고자 입력함.
model.add(Dense(vocab_size,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=num_epoch,verbose=1)

Epoch 1/100
244/244 [==============================] - 2s 4ms/step - loss: 7.6459 - accuracy: 0.0288
Epoch 2/100
244/244 [==============================] - 1s 4ms/step - loss: 7.1166 - accuracy: 0.0306
Epoch 3/100
244/244 [==============================] - 1s 4ms/step - loss: 6.9722 - accuracy: 0.0343
Epoch 4/100
244/244 [==============================] - 1s 4ms/step - loss: 6.8435 - accuracy: 0.0408
Epoch 5/100
244/244 [==============================] - 1s 4ms/step - loss: 6.6864 - accuracy: 0.0454
Epoch 6/100
244/244 [==============================] - 1s 4ms/step - loss: 6.5102 - accuracy: 0.0472
Epoch 7/100
244/244 [==============================] - 1s 4ms/step - loss: 6.3179 - accuracy: 0.0532
Epoch 8/100
244/244 [==============================] - 1s 4ms/step - loss: 6.1192 - accuracy: 0.0588
Epoch 9/100
244/244 [==============================] - 1s 4ms/step - loss: 5.9200 - accuracy: 0.0634
Epoch 10/100
244/244 [==============================] - 1s 4ms/step - loss: 5.7295 - accura

#### 문장을 생성하는 함수 sentence_generation을 만들어서 다음 시점의 단어를 예측하고 문장을 만들어보자.

In [52]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''
    
    # current_word를 계속 쌓아 나가서 다음 단어를 예측하는 것을 재귀적으로 n번 반복하는 느낌.
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') # max_len-1은 X,y로 분리되었기 때문에 -1을 한 것.

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0) # shape : (1,vocab_size)
        result = np.argmax(result, axis=1) # 가장 확률이 높은 index를 추출.
        
        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # current_word를 {' ' + 예측 단어}를 추가해서 저장하고 다음 입력으로 사용
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장해서 최종 출력에 사용하기.
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

print(sentence_generation(model, tokenizer, 'i', 10)) 

i disapprove of school vouchers can i still apply for them


근데 i 하나만 주어지고 나머지를 예측하는게 좀 웃기긴 하다. i 다음으로 나올 수 있는게 얼마나 많은데.. 그냥 지금은 이런 느낌으로 학습을 진행하고 예측한다고만 생각하자.